In [24]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json

# EXTRACT CHICAGO CRIME DATA (CSV)

In [25]:
# Input Crime File (CSV)
crime_df = pd.read_csv('Resources/chicago_crime.csv')
crime_df.head(2)

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,10801862,JA100930,2017-01-01,079XX S RACINE AVE,281,CRIM SEXUAL ASSAULT,NON-AGGRAVATED,RESIDENCE,False,False,...,21.0,71.0,2,1169694.0,1852368.0,2017,2/14/2017 15:49,41.750397,-87.653735,"(41.750396912, -87.653735437)"
1,11181755,JA554736,2017-01-01,083XX S PEORIA ST,266,CRIM SEXUAL ASSAULT,PREDATORY,ABANDONED BUILDING,False,False,...,21.0,71.0,2,1171744.0,1849640.0,2017,2/10/2018 15:50,41.742866,-87.646303,"(41.742866241, -87.646303123)"


# TRANSFORM CHICAGO CRIME DATA

In [49]:
#Drop crime_df columns: ID, Case Number, Block, IUCR, Domestic, Ward, X Coordinate, Y Coordinate, Year, Updated On, Latitude, Longitude, Location
crime = crime_df.drop(columns=['ID', 'Case Number', 'Block', 'IUCR', 'Domestic', 'Ward', 'X Coordinate',
                         'Y Coordinate', 'Year', 'Updated On', 'Latitude', 'Longitude', 'Location',
                         'District', 'Community Area', 'FBI Code', 'Arrest', 'Beat', 'Description', 'Location Description'], axis=1)
#Strip off the dates
dates=[]
for row in crime['Date']:
    dates.append(row[:7])
crime['Dates'] = dates
crime = crime.drop(['Date'], axis=1)

# column replace using .loc, count daily incidents
crime_robbery = crime.loc[crime['Primary Type'] == 'ROBBERY']
crime_robbery = crime_robbery.groupby('Dates').count()
crime_robbery.rename(columns={'Primary Type': 'Robbery'}, inplace=True)


crime_assault = crime.loc[crime['Primary Type'] == 'ASSAULT']
crime_assault = crime_assault.groupby('Dates').count()
crime_assault.rename(columns={'Primary Type': 'Assault'}, inplace=True)

crime_sassault = crime.loc[crime['Primary Type'] == 'CRIM SEXUAL ASSAULT']
crime_sassault = crime_sassault.groupby('Dates').count()
crime_sassault.rename(columns={'Primary Type': 'Sexual Assault'}, inplace=True)

# merge the dataframes
crime_output = pd.merge(crime_robbery, crime_assault,  on='Dates')
crime_output = pd.merge(crime_output, crime_sassault, on='Dates')
crime_output.head()

# Write to CSV
crime_output.to_csv('Resources/chicago_crime_ouput.csv' ,encoding='utf-8')

# EXTRACT CHICAGO WEATHER DATA (JSON)

In [27]:
#Import Weather JSON file

weather_df = pd.read_json('Resources/chicago_weather.json', orient='columns')
weather_df.head()

,city_id,clouds,dt,dt_iso,main,rain,snow,weather,wind
0,4887398,{'all': 90},1483228800,2017-01-01 00:00:00 +0000 UTC,"{'temp': 274.06, 'temp_min': 273.15, 'temp_max...",NaN,NaN,"[{'id': 804, 'main': 'Clouds', 'description': ...","{'speed': 4, 'deg': 300}"
1,4887398,{'all': 40},1483232400,2017-01-01 01:00:00 +0000 UTC,"{'temp': 272.94, 'temp_min': 271.15, 'temp_max...",NaN,NaN,"[{'id': 802, 'main': 'Clouds', 'description': ...","{'speed': 4, 'deg': 290}"
2,4887398,{'all': 1},1483236000,2017-01-01 02:00:00 +0000 UTC,"{'temp': 272.14, 'temp_min': 271.15, 'temp_max...",NaN,NaN,"[{'id': 800, 'main': 'Clear', 'description': '...","{'speed': 3, 'deg': 270}"
3,4887398,{'all': 1},1483239600,2017-01-01 03:00:00 +0000 UTC,"{'temp': 270.97, 'temp_min': 270.15, 'temp_max...",NaN,NaN,"[{'id': 800, 'main': 'Clear', 'description': '...","{'speed': 3, 'deg': 250}"
4,4887398,{'all': 1},1483243200,2017-01-01 04:00:00 +0000 UTC,"{'temp': 270.14, 'temp_min': 269.15, 'temp_max...",NaN,NaN,"[{'id': 800, 'main': 'Clear', 'description': '...","{'speed': 1, 'deg': 220}"


# TRANSFORM CHICAGO WEATHER DATA

In [28]:
weather = weather_df.drop(columns=['city_id', 'weather', 'clouds', 'dt'], axis=1)
weather.rename(columns={'dt_iso': 'date', 'main': 'temp'}, inplace=True)

In [50]:
# Extract date, Max Temp, Min Temp, Windspeed

#Strip date
dates=[]
for row in weather['date']:
    dates.append(row[:7])

#Strip max and min temperatures, convert from Kelvin to Fahrenheit
max_temp =[]
min_temp =[]
for row in weather['temp']:
    max_temp.append(9/5 * (row['temp_max']-273)+32)
    min_temp.append(9/5 * (row['temp_min']-273)+32)

#Strip wind category
winds = []
for row in weather['wind']:
    winds.append(row['speed'])

#Add new lists to weather dataframe. Convert Kelvin temperature to Fahrenheit
weather['dates'] = dates
weather['max_temp'] = max_temp
weather['min_temp'] = min_temp
weather['winds'] = winds

#Drop old columns from dataframe
weather_clean = weather.drop(columns=['date', 'temp', 'rain', 'snow', 'wind'], axis=1)

#Group by daily averages
weather_daily = weather_clean.groupby(['dates']).mean()
weather_daily.head()

# Write to CSV
weather_daily.to_csv('Resources/chicago_weather_ouput.csv' ,encoding='utf-8')